# L3b: Classification of Clinical Breast Cancer Samples
Linear regression can be adapted for classification tasks by transforming the continuous output of the linear regression model directly to a class designation, e.g., $\sigma:\mathbb{R}\rightarrow\{-1,+1\}$ or into a probability using an output function $\sigma:\mathbb{R}\rightarrow\mathbb{R}$ and applying a threshold to categorize predictions into discrete classes. Let's take a look at two examples of these strategies:

* [The Perceptron (Rosenblatt, 1957)](https://en.wikipedia.org/wiki/Perceptron) is a simple yet powerful algorithm used in machine learning for binary classification tasks. It operates by _incrementally_ learning a linear decision boundary (linear regression model) that separates two classes based on input features by directly mapping the continuous output to a class such as $\sigma:\mathbb{R}\rightarrow\{-1,+1\}$, where the output function is $\sigma(\star) = \text{sign}(\star)$.
* [Logistic regression](https://en.wikipedia.org/wiki/Logistic_regression#) is a statistical method used in machine learning for binary classification tasks using the [logistics function](https://en.wikipedia.org/wiki/Logistic_function) as the transformation function. Applying the logistic function transforms the output of a linear regression model into a probability, enabling effective decision-making in various applications. We'll consider this approach next time.

### Perceptron
[The Perceptron (Rosenblatt, 1957)](https://en.wikipedia.org/wiki/Perceptron) takes the (scalar) output of a linear regression model $y_{i}\in\mathbb{R}$ and then transforms it using the $\sigma(\star) = \text{sign}(\star)$ function to a discrete set of values representing categories, e.g., $\sigma:\mathbb{R}\rightarrow\{-1,1\}$ in the binary classification case. 
* Suppose there exists a data set
$\mathcal{D} = \left\{(\mathbf{x}_{1},y_{1}),\dotsc,(\mathbf{x}_{n},y_{n})\right\}$ with $n$ _labeled_ examples, where each example has been labeled by an expert, i.e., a human to be in a category $\hat{y}_{i}\in\{-1,1\}$, given the $m$-dimensional feature vector $\mathbf{x}_{i}\in\mathbb{R}^{m}$. 
* [The Perceptron](https://en.wikipedia.org/wiki/Perceptron) _incrementally_ learns a linear decision boundary between _two_ classes of possible objects (binary classification) in $\mathcal{D}$ by repeatedly processing the data. During each pass, a regression parameter vector $\mathbf{\beta}$ is updated until it makes no more than a specified number of mistakes. 

[The Perceptron](https://en.wikipedia.org/wiki/Perceptron) computes the estimated label $\hat{y}_{i}$ for feature vector $\hat{\mathbf{x}}_{i}$ using the $\texttt{sign}:\mathbb{R}\to\{-1,1\}$ function:
$$
\begin{equation*}
    \hat{y}_{i} = \texttt{sign}\left(\hat{\mathbf{x}}_{i}^{\top}\cdot\beta\right)
\end{equation*}
$$
where $\beta=\left(w_{1},\dots,w_{n}, b\right)$ is a column vector of (unknown) classifier parameters, $w_{j}\in\mathbb{R}$ corresponding to the importance of feature $j$ and $b\in\mathbb{R}$ is a bias parameter, the features $\hat{\mathbf{x}}^{\top}_{i}=\left(x^{(i)}_{1},\dots,x^{(i)}_{m}, 1\right)$ are $p = m+1$-dimensional (row) vectors (features augmented with bias term), and $\texttt{sign}(z)$ is the function:
$$
\begin{equation*}
    \texttt{sign}(z) = 
    \begin{cases}
        1 & \text{if}~z\geq{0}\\
        -1 & \text{if}~z<0
    \end{cases}
\end{equation*}
$$
__Hypothesis__: If data set $\mathcal{D}$ is linearly separable, the Perceptron will _incrementally_ learn a separating hyperplane in a finite number of passes through the data set $\mathcal{D}$. However, if the data set $\mathcal{D}$ is not linearly separable, the Perceptron may not converge. Check out a [perceptron pseudo-code here!](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-3/L3a/docs/Notes.pdf)
* __Training__: Our Perceptron implementation [based on pseudo-code](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-3/L3a/docs/Notes.pdf) stores problem information in [a `MyPerceptronClassificationModel` instance, which holds the (initial) parameters and other data](src/Types.jl) required by the problem. We then _learn_ the model parameters [using the `learn(...)` method](src/Compute.jl), which takes the training features array `X,` the training labels vector `y`, and the problem instance and returns back an updated problem instance holding the updated parameters.
* __Challenge__: We've never seen this data and have no idea if it's linearly separable. Thus, we have no theoretical guarantee that [the Perceptron](https://en.wikipedia.org/wiki/Perceptron) will work. Let's load the dataset, do some preprocessing, and then explore the performance of [the Perceptron](https://en.wikipedia.org/wiki/Perceptron) on this data.
* Let's compare our results to the study [Sidey-Gibbons, J., Sidey-Gibbons, C. Machine learning in medicine: a practical introduction. BMC Med Res Methodol 19, 64 (2019). https://doi.org/10.1186/s12874-019-0681-4](https://rdcu.be/d5NjG), that used the same dataset.

## Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem.

In [3]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-5820-SP25/CHEME-5820-Labs-Spring-2025/labs/week-3/L3b`
    Updating `~/Desktop/julia_work/CHEME-5820-SP25/CHEME-5820-Labs-Spring-2025/labs/week-3/L3b/Project.toml`
  [336ed68f] + CSV v0.10.15
  [a93c6f00] + DataFrames v1.7.0
  [31c24e10] + Distributions v0.25.117
  [5789e2e9] + FileIO v1.16.6
  [91a5bcdd] + Plots v1.40.9
  [08abe8d2] + PrettyTables v2.4.0
  [10745b16] + Statistics v1.11.1
  [2913bbd2] + StatsBase v0.34.4
  [f3b207a7] + StatsPlots v0.15.7
  [37e2e46d] ~ LinearAlgebra ⇒ v1.11.0
    Updating `~/Desktop/julia_work/CHEME-5820-SP25/CHEME-5820-Labs-Spring-2025/labs/week-3/L3b/Manifest.toml`
  [621f4979] + AbstractFFTs v1.5.0
  [79e6a3ab] + Adapt v4.1.1
  [66dad0bd] + AliasTables v1.1.3
  [7d9fca2a] + Arpack v0.5.4
  [13072b0f] + AxisAlgorithms v1.1.0
  [d1d4a3ce] + BitFlags v0.1.9
  [336ed68f] + CSV v0.10.15
  [d360d2e6] + ChainRulesCore v1.25.1
  [aaaa29a8] + Clustering v0.15.8
  [944b1d66] + CodecZlib v0.7.6
  [35d6a980] +

### Data
In this lab, we'll use [the Perceptron (Rosenblatt, 1957)](https://en.wikipedia.org/wiki/Perceptron) to classify clinical Breast Cancer samples taken from the University of Wisconsin. 
* __Description__: The breast cancer dataset developed by [Wolberg, W. (1990)](https://doi.org/10.24432/C5HP4Z) was obtained from the University of Wisconsin Hospitals, Madison, from [Dr. William H. Wolberg](https://pages.cs.wisc.edu/~olvi/uwmp/cancer.html), and is available [from the UCI dataset archive](https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original). It contains `699` instances, with `9` clinical features and a class label `{benign | malignant}.`

In [5]:
df = CSV.read(joinpath(_PATH_TO_DATA, "breast-cancer-wisconsin.csv"), DataFrame)

Row,id,ClumpThickness,UniformityCellSize,UniformityCellShape,MarginalAdhesion,SingleEpithelialCellSize,BareNuclei,BlandChromatin,NormalNucleoli,Mitoses,Class
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1000025,5,1,1,1,2,1,3,1,1,2
2,1002945,5,4,4,5,7,10,3,2,1,2
3,1015425,3,1,1,1,2,2,3,1,1,2
4,1016277,6,8,8,1,3,4,3,7,1,2
5,1017023,4,1,1,3,2,1,3,1,1,2
6,1017122,8,10,10,8,7,10,9,7,1,4
7,1018099,1,1,1,1,2,10,3,1,1,2
8,1018561,2,1,2,1,2,1,3,1,1,2
9,1033078,2,1,1,1,2,1,1,1,5,2


__Data wrangling__: The `Class` label is not in the form of $\{-1,1\}$ that the perceptron expects, so let's transform the original data where we map $2\rightarrow{-1}$ and $4\rightarrow{1}$. We'll save the transformed data in the `dataset::DataFrame` variable. In our transformed dataset the `-1` labels is _not cancer_ while the label `1` denotes _cancer_.

In [7]:
dataset = let

    number_of_examples = nrow(df);
    for i ∈ 1:number_of_examples
        c = df[i,:Class];
        if (c == 2)
            df[i,:Class] = -1 # not cancer
        elseif (c == 4)
            df[i,:Class] = 1 # cancer
        end
    end
    df
end

Row,id,ClumpThickness,UniformityCellSize,UniformityCellShape,MarginalAdhesion,SingleEpithelialCellSize,BareNuclei,BlandChromatin,NormalNucleoli,Mitoses,Class
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1000025,5,1,1,1,2,1,3,1,1,-1
2,1002945,5,4,4,5,7,10,3,2,1,-1
3,1015425,3,1,1,1,2,2,3,1,1,-1
4,1016277,6,8,8,1,3,4,3,7,1,-1
5,1017023,4,1,1,3,2,1,3,1,1,-1
6,1017122,8,10,10,8,7,10,9,7,1,1
7,1018099,1,1,1,1,2,10,3,1,1,-1
8,1018561,2,1,2,1,2,1,3,1,1,-1
9,1033078,2,1,1,1,2,1,1,1,5,-1


Next, let's (randomly) partition the clinical data into `training` and `test` sets. 
* __Training data__: Training datasets are collections of labeled data used to teach machine learning models, allowing these tools to learn patterns and relationships within the data. In our case, we'll use the training data to estimate the classifier parameters $\beta$.
* __Test data__: Test datasets, on the other hand, are separate sets of labeled data used to evaluate the performance of trained models on unseen examples, providing an unbiased assessment of the _model's generalization capabilities_.

In [9]:
training, test = let

    number_of_training_examples = 456; # from Sidey-Gibbons, 2019
    D = Matrix(dataset);
    number_of_features = size(D,2); # number of cols of housing data
    number_of_examples = size(D,1); # number of rows of housing data
    full_index_set = range(1,stop=number_of_examples,step=1) |> collect |> Set;
    
    # build index sets for training and testing
    training_index_set = Set{Int64}();
    should_stop_loop = false;
    while (should_stop_loop == false)
        i = rand(1:number_of_examples);
        push!(training_index_set,i);

        if (length(training_index_set) == number_of_training_examples)
            should_stop_loop = true;
        end
    end
    test_index_set = setdiff(full_index_set,training_index_set);

    # build the test and train datasets -
    training = D[training_index_set |> collect,:];
    test = D[test_index_set |> collect,:];

    # return
    training, test
end;

In [10]:
test

243×11 Matrix{Int64}:
 1115762   3   1   1   1   2   1   1   1   1  -1
 1147748   5  10   6   1  10   4   4  10  10   1
 1204898   6   1   1   1   2   1   3   1   1  -1
  763235   3   1   1   1   2   1   2   1   2  -1
 1293966   4   1   1   1   2   1   1   1   1  -1
 1365075   4   1   4   1   2   1   1   1   1  -1
 1056784   3   1   1   1   2   1   2   1   1  -1
 1350319   5   7   4   1   6   1   7  10   3   1
 1096800   6   6   6   9   6   0   7   8   1  -1
 1118039   5   3   4   1   8  10   4   9   1   1
  704097   1   1   1   1   1   1   2   1   1  -1
  466906   1   1   1   1   2   1   1   1   1  -1
  769612   3   1   1   2   2   1   1   1   1  -1
       ⋮                   ⋮                   ⋮
 1099510  10   4   3   1   3   3   6   5   2   1
  635844   8   4  10   5   4   4   7  10   1   1
  560680   1   1   1   1   2   1   1   1   1  -1
 1169049   7   3   4   4   3   3   3   2   7   1
 1202253   5   1   1   1   2   1   1   1   1  -1
 1198641   3   1   1   1   2   1   3   1   1  -

## Task 1: Build a Classification Model and Learn the Parameters
In this task, we'll build a model of our classification problem and train the model using a learning method. 
* Our Perceptron implementation [based on our pseudocode](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-3/L3a/docs/Notes.pdf) stores problem information in [a `model::MyPerceptronClassificationModel` instance, which holds the (initial) parameters and other data](src/Types.jl) required by the problem. 

In [12]:
model = let

    # How many features do we have?
    D = training; # let's look at the training data
    number_of_features = size(D,2) - 1; # why minus one?
    
    # build a model
    model = build(MyPerceptronClassificationModel, (
        parameters = ones(number_of_features),
        mistakes = 0 # willing to like with m mistakes
    ));

    model;
end;

Next, we then _learn_ the model parameters [using the `learn(...)` method](src/Compute.jl), which takes the training features array `X,` the training labels vector `y`, and the problem instance and returns an updated problem instance holding the updated parameters. 

In [14]:
trainedmodel = let

    D = training; # what dataset are we going to use?
    number_of_examples = size(D,1); # how many examples do we have (rows)
    number_of_features = size(D,2) - 1; # how many features do we have (cols)?
    X = [D[:,2:end-1] ones(number_of_examples)]; # features, what??
    y = D[:,end]; # output: this is the target data (label)
    
    # train the model -
    trainedmodel = learn(X,y,model, maxiter = 1000, verbose = true);

    # return
    trainedmodel;
end;

Stopped after number of iterations: 1000. We have number of errors: 14


## Task 2: Classify the test data and compute the Confusion matrix
In this task, we'll use the updated `model::MyPerceptronClassificationModel` instance (that has learned some parameters from the `training` data) and test how well we classify data that we have never seen, i.e., how well we classify the `test` dataset.
* __Inference__: We run the classification operation on the (unseen) test data [using the `classify(...)` method](src/Compute.jl). This method takes a feature array `X` and the (trained) model instance. It returns the estimated labels. We store the actual (correct) label in the `y::Array{Int64,1}` vector, while the model predicted label is stored in the `ŷ::Array{Int64,1}` array.
* __Performance__: Once the Perceptron (or any binary classifier) has converged, we can evaluate the binary classifier's performance using various metrics. The central idea is to compare the predicted labels $\hat{y}_{i}$ to the actual labels $y_{i}$ in the data set $\mathcal{D}$. 
Various metrics can be used to evaluate the performance of a binary classifier, but they all start with computing the confusion matrix.

In [16]:
ŷ,y = let

    D = test; # what dataset are going to use?
    number_of_examples = size(D,1); # how many examples do we have (rows)
    number_of_features = size(D,2) - 1; # how many features do we have (cols)?
    X = [D[:,2:end-1] ones(number_of_examples)]; # features: need to add a 1 to each row (for bias), after removing the label
    y = D[:,end]; # output: this is the *actual* target data (label)

    # compute the estimated labels -
    ŷ = classify(X,model)

    # return -
    ŷ,y
end;

### Confusion matrix
The confusion matrix is a $2\times{2}$ matrix that contains four entries: true positive (TP), false positive (FP), true negative (TN), and false negative (FN). [Click me for a confusion matrix schematic!](https://github.com/varnerlab/CHEME-5820-Labs-Spring-2025/blob/main/labs/week-3/L3b/figs/Fig-BinaryConfusionMatrix.pdf)
* __The true positive (TP)__ entry in the confusion matrix is the number of positive examples that were correctly classified as positive.
* __The false negative (FN)__ entry indicates the number of positive examples the model incorrectly classified as negative.
* __The false positive (FP)__ entry is the number of negative examples that were incorrectly classified as positive by the model.
* __The true negative (TN)__ entry is the number of negative examples that were correctly classified as negative by the model.

The confusion matrix calculates various performance metrics, including accuracy, precision, recall, and the F1 score.

In [18]:
overall_misclassified_percentage = let
    
    number_of_test_examples = length(ŷ);
    error_counter = 0;

    for i ∈ 1:number_of_test_examples
        if (ŷ[i] != y[i])
            error_counter += 1;
        end
    end
    
    error_counter/number_of_test_examples
end

0.03292181069958848

__Accuracy__: Accuracy is a fundamental metric used to evaluate the performance of binary classifiers. Accuracy is mathematically defined as the ratio of correctly predicted instances (both true positives and true negatives) to the total number of cases. 
$$
\begin{equation*}
\text{Accuracy} = \frac{N_{+}+N_{-}}{N_{T}}
\end{equation*}
$$
where $N_{+}$ (or $N_{-}$) denotes the number of true positive (or negative) classifications, i.e., the number of times the classifier estimates the proper label, and $N_{T}$ denotes the total number of prediction samples.

In [20]:
accuracy = let

    number_of_test_examples = length(ŷ);
    correct_counter = 0;

    for i ∈ 1:number_of_test_examples
        if (ŷ[i] == y[i])
            correct_counter += 1;
        end
    end
    
    correct_counter/number_of_test_examples
end

0.9670781893004116

__Specificity__: The specificity, also known as the True Negative Rate (TNR), measures how well the model distinguishes between the positive and negative classes. The specificity is defined as:
$$
\begin{equation*}
\text{Specificity} = \frac{N_{-,-}}{N_{-,-}+N_{+,-}}
\end{equation*}
$$
where $N_{-,-}$ denotes the number of actual `negative` samples that were classified as `negative,` and $N_{+,-}$ denotes the number of (actual) `positive` samples that were (mis)classified as `negative` (false negative). The denominator is the total number of samples predicted to be `negative.`

In [22]:
specificity = let

    number_of_test_examples = length(ŷ);
    total_number_of_predicted_negatives = findall(c-> c == -1, ŷ) |> length;
    counter = 0;

    for i ∈ 1:number_of_test_examples
        if (y[i] == -1 && ŷ[i] == -1) # N(-1,-1)
            counter += 1;
        end
    end

    counter/total_number_of_predicted_negatives
end

0.9938271604938271

__Precision__: The precision measures the proportion of true positive predictions among all positive predictions. Precision is defined as:
$$
\begin{equation*}
\text{Precision} = \frac{N_{+,+}}{N_{+,+} + N_{-,+}}
\end{equation*}
$$
where $N_{+,+}$  denotes the number of actual `positive` classifications in which the model correctly predicts `positive,` and $N_{-,+}$ denotes the number of actual `negative` samples that are predicted by the model to be `positive` (false positive). The denominator is the total number of `positive` predictions.

In [24]:
precision = let

    number_of_test_examples = length(ŷ);
    total_number_of_predicted_positives = findall(c-> c == 1, ŷ) |> length;
    counter = 0;

    for i ∈ 1:number_of_test_examples
        if (y[i] == 1 && ŷ[i] == 1)
            counter += 1;
        end
    end

    counter/total_number_of_predicted_positives
end

0.9135802469135802